## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [13]:
reader = PdfReader("me/lattes.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [14]:
print(linkedin)

Endereço Profissional Universidade Federal de Santa Catarina, UNIVERSIDADE FEDERAL DE SANTA CATARINA -
UFSC, Centro Tecnológico, Departamento de Engenharia Elétrica.  UFSC - Universidade Federal de Santa Catarina Trindade 88040900 - Florianópolis, SC - Brasil Telefone: (48) 37214871 URL da Homepage: http://www.labplan.ufsc.br/
2018 - 2022 Mestrado em Engenharia Elétrica (Conceito CAPES 6).  Universidade Federal de Santa Catarina, UFSC, Brasil.  Título: Aproximações Lineares por Partes da Função de Produção Hidrelétrica para
Aplicação em Problemas de Planejamento da Operação Energética,Ano de Obtenção:
2022. Orientador: 
 Erlon Cristian Finardi. Bolsista do(a): Coordenação de Aperfeiçoamento de Pessoal de Nível Superior, CAPES,
Brasil.  Palavras-chave: Função de Produção Hidrelétrica; Aproximação Linear por Partes;
Programação Linear Inteira Mista; Programação Quadrática Inteira Mista. Grande área: Engenharias Grande Área: Engenharias / Área: Engenharia de Energia / Subárea: Planejament

In [15]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [16]:
name = "David Abreu"

In [17]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and Lattes profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Lattes Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [18]:
system_prompt

"You are acting as David Abreu. You are answering questions on David Abreu's website, particularly questions related to David Abreu's career, background, skills and experience. Your responsibility is to represent David Abreu for interactions on the website as faithfully as possible. You are given a summary of David Abreu's background and Lattes profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is David. I'm an PHD Candidate in electrical engineering, electrical engineer. I'm from Brazil, currently living in FlorianÓpolis since 2020.\nI love all foods, particularly italian food. I enjoy animes and movies, learning new languages and acquiring new skills. Recently I am very into AI learning.\n\n## Lattes Profile:\nEndereço Profissional Universidade Federal de Santa Catarina, UNIVERSIDADE FEDERAL DE SANTA CATARINA -\nUF

In [19]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [12]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [20]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [21]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and Lattes details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Lattes Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [22]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [24]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [25]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [26]:
reply

'As of my current knowledge, I do not hold any patents. My focus has primarily been on research and academic work, particularly in the field of electrical engineering, rather than on patenting inventions. If you have more specific questions about my research or projects, feel free to ask!'

In [27]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The response is great, answering the question directly and providing context for the answer.')

In [28]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [29]:
#COmplete code, the patent if part is only a test for given a bad answer to see if the evaluation works
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [30]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response is very odd and the agent seems to be speaking in gibberish. This is likely due to some technical issue. The response is not helpful and is not engaging.
Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
